In [ ]:
!pip install tinyimagenet
!pip install torchvision
!pip install seaborn
!pip install torchmetrics
!pip install poutyne

In [ ]:
from tinyimagenet import TinyImageNet
from pathlib import Path
import torch
from torchvision import models

In [ ]:
from torch import nn
import torchvision


def block(f_in,f_out,kernel_size=(3,3),padding=1):
    return torch.nn.Sequential(
        nn.Conv2d(f_in, f_out, kernel_size, padding=padding),
        nn.ReLU(),
        nn.BatchNorm2d(f_out),
        nn.MaxPool2d(2),
    )


classes = 200
channels = 3

f= 128
fc = 1024

conv = torch.nn.Sequential(
    nn.Conv2d(channels, f, (5,5), stride=(2,2), padding=3),
    nn.ReLU(),
    block(f,f), #32x32
    block(f,2*f), #16x16
    block(2*f,4*f), #8x8
    block(4*f,8*f), #4x4
)
pixels = (64*64) / (2 * (2**5))
linear_size= pixels*8*f

model = torch.nn.Sequential(
    torchvision.transforms.Normalize(TinyImageNet.mean,TinyImageNet.std),
    conv,
    nn.Flatten(),
    nn.Linear(linear_size, fc),
    nn.BatchNorm1d(fc),
    nn.Linear(fc, classes),
)
print(model)


In [ ]:
import poutyne


device = torch.device("cuda")
def get_dataset(split): return TinyImageNet(Path("~/.torchvision/tinyimagenet/"),split=split,imagenet_idx=False)

train_dataset = get_dataset("train")
val_dataset = get_dataset("val")


trainer = poutyne.Model(
        model,
        'sgd',
        'cross_entropy',
        batch_metrics=['accuracy',poutyne.TopKAccuracy(5)],
        epoch_metrics=['accuracy'],
        device=device
)


trainer.fit_dataset(train_dataset,valid_dataset=val_dataset,batch_size=8,epochs=5)